In [32]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm import tqdm
import numpy as np
import pickle

# Load Data

In [33]:
def load_vectors(pos_file, neg_file):
    """
    Load positive and negative vectors, combine and shuffle them.
    
    Parameters:
    pos_file: str
        Path to file containing positive vectors.
    neg_file: str
        Path to file containing negative vectors.
    
    Returns:
    X: np.array
        Combined and shuffled vectors.
    y: np.array
        Labels for the vectors.
    """
    # Read files
    pos_vectors = np.loadtxt(pos_file)
    neg_vectors = np.loadtxt(neg_file)
    
    # Create labels
    pos_labels = np.ones(len(pos_vectors))
    print(pos_labels)
    neg_labels = -np.ones(len(neg_vectors))
    print(neg_labels)
    
    # Combine data
    X = np.vstack((pos_vectors, neg_vectors))
    y = np.concatenate((pos_labels, neg_labels))
    
    # Shuffle
    indices = np.arange(len(X))
    np.random.shuffle(indices)
    
    return X[indices], y[indices]

In [34]:
X, y = load_vectors("data/twitter-datasets/train_pos_embedding.txt", "data/twitter-datasets/train_neg_embedding.txt")

[1. 1. 1. ... 1. 1. 1.]
[-1. -1. -1. ... -1. -1. -1.]


In [35]:
def train_neural_network(X, y, hidden_layers=(64, 32), max_iter=200):
    """Train MLPClassifier with progress tracking"""
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    model = MLPClassifier(
        hidden_layer_sizes=hidden_layers,
        max_iter=max_iter,
        random_state=42,
        solver='adam',
        verbose=True
    )
    
    # Training with progress bar
    print("Training neural network...")
    with tqdm(total=max_iter) as pbar:
        def update_progress(iter_num, loss):
            pbar.update(1)
            pbar.set_description(f'Loss: {loss:.4f}')
        
        # Set custom callback
        model._callback = update_progress
        model.fit(X_train, y_train)
    
    # Evaluation
    y_pred = model.predict(X_test)
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    
    return model

In [36]:
def predict_sentiment_nn(model, tweet_vector):
    """Predict sentiment for a single tweet vector"""
    return model.predict([tweet_vector])[0]

# Train model

In [37]:
model = train_neural_network(X, y)

Training neural network...


  0%|          | 0/200 [00:00<?, ?it/s]

Iteration 1, loss = 0.66421367
Iteration 2, loss = 0.65212917
Iteration 3, loss = 0.64706856
Iteration 4, loss = 0.64448409
Iteration 5, loss = 0.64252011
Iteration 6, loss = 0.64063967
Iteration 7, loss = 0.63854473
Iteration 8, loss = 0.63784463
Iteration 9, loss = 0.63665243
Iteration 10, loss = 0.63526054
Iteration 11, loss = 0.63426296
Iteration 12, loss = 0.63357941
Iteration 13, loss = 0.63257226
Iteration 14, loss = 0.63169847
Iteration 15, loss = 0.63130753
Iteration 16, loss = 0.63021910
Iteration 17, loss = 0.62926693
Iteration 18, loss = 0.62863642
Iteration 19, loss = 0.62809115
Iteration 20, loss = 0.62798907
Iteration 21, loss = 0.62701000
Iteration 22, loss = 0.62641326
Iteration 23, loss = 0.62641157
Iteration 24, loss = 0.62532381
Iteration 25, loss = 0.62539447
Iteration 26, loss = 0.62427723
Iteration 27, loss = 0.62329792
Iteration 28, loss = 0.62317620
Iteration 29, loss = 0.62283862
Iteration 30, loss = 0.62197917
Iteration 31, loss = 0.62195319
Iteration 32, los

/Users/nicolas/Library/Python/3.9/lib/python/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
  0%|          | 0/200 [01:46<?, ?it/s]

Iteration 200, loss = 0.59675117

Classification Report:
              precision    recall  f1-score   support

        -1.0       0.65      0.60      0.63     19968
         1.0       0.63      0.68      0.65     20032

    accuracy                           0.64     40000
   macro avg       0.64      0.64      0.64     40000
weighted avg       0.64      0.64      0.64     40000



In [38]:
# Save model
with open("average_NN_model.pkl", "wb") as f:
    pickle.dump(model, f)

AttributeError: Can't pickle local object 'train_neural_network.<locals>.update_progress'